In [1]:
import os
import gzip
from Bio import SeqIO
import pandas as pd

# Directories
dna_dir = "data/dna_sequences/"
gff3_dir = "data/gff3_files/"

# Helper: extract chromosome ID from filename
def extract_chr_id(filename):
    # Works for names like: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.1.gff3
    parts = filename.split("chromosome.")
    return parts[1].split(".")[0] if len(parts) > 1 else None

# Collect and sort all relevant files
fasta_files = sorted([
    os.path.join(dna_dir, f) for f in os.listdir(dna_dir)
    if f.lower().endswith(".fa.gz") or f.lower().endswith(".fasta")
])
gff3_files = sorted([
    os.path.join(gff3_dir, f) for f in os.listdir(gff3_dir)
    if f.lower().endswith(".gff3")
])

# Create a mapping: chromosome_id -> file path
fasta_map = {extract_chr_id(os.path.basename(f)): f for f in fasta_files}
gff3_map = {extract_chr_id(os.path.basename(f)): f for f in gff3_files}

# Function to parse GFF3 and extract gene features
def parse_gff3(gff3_file):
    genes = []
    with open(gff3_file, encoding="utf-8") as f:
        for line in f:
            if line.startswith("#"):
                continue
            parts = line.strip().split("\t")
            if len(parts) < 9:
                continue
            if parts[2] == "gene":
                start = int(parts[3]) - 1  # Convert from 1-based to 0-based
                end = int(parts[4])
                strand = parts[6]
                attr = parts[8]
                gene_id = "NA"
                for field in attr.split(";"):
                    if field.startswith("ID="):
                        gene_id = field.split("=")[1]
                        break
                genes.append((start, end, strand, gene_id))
    return genes

# List to collect all extracted gene data
gene_sequences = []

# Match and process chromosome-wise
for chr_id in sorted(set(fasta_map.keys()) & set(gff3_map.keys())):
    fasta_path = fasta_map[chr_id]
    gff_path = gff3_map[chr_id]
    
    print(f"Processing Chromosome {chr_id}: {os.path.basename(fasta_path)} & {os.path.basename(gff_path)}")

    # Read DNA sequence (gz or regular)
    if fasta_path.endswith(".gz"):
        with gzip.open(fasta_path, "rt", encoding="utf-8") as f:
            record = next(SeqIO.parse(f, "fasta"))
    else:
        with open(fasta_path, "r", encoding="utf-8") as f:
            record = next(SeqIO.parse(f, "fasta"))

    chrom_seq = record.seq
    chrom_name = record.id

    # Parse GFF3 gene entries
    genes = parse_gff3(gff_path)

    # Extract and store gene sequences
    for start, end, strand, gene_id in genes:
        gene_seq = chrom_seq[start:end]
        if strand == "-":
            gene_seq = gene_seq.reverse_complement()
        gene_sequences.append({
            "gene_id": gene_id,
            "chromosome": chrom_name,
            "chromosome_id": chr_id,
            "start": start,
            "end": end,
            "strand": strand,
            "sequence": str(gene_seq)
        })

# Convert to DataFrame
df_genes = pd.DataFrame(gene_sequences)

# Preview the result
print("\nExtracted Gene Sequences:\n", df_genes.head())


Processing Chromosome 1: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.1.fa.gz & Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.1.gff3
Processing Chromosome 10: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.10.fa.gz & Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.10.gff3
Processing Chromosome 2: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.2.fa.gz & Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.2.gff3
Processing Chromosome 3: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.3.fa.gz & Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.3.gff3
Processing Chromosome 4: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.4.fa.gz & Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.4.gff3
Processing Chromosome 5: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.5.fa.gz & Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.5.gff3
Processing Chromosome 6: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.6.fa.gz & Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.6.gff3
Processing Chromosome 7:

In [2]:
import pandas as pd

# Define the window size around TSS (e.g., 10,000 bp on either side)
WINDOW_SIZE = 10000

# Calculate TSS and window region
def compute_tss_window(df_genes):
    windows = []
    for _, row in df_genes.iterrows():
        if row["strand"] == "+":
            tss = row["start"]
        else:
            tss = row["end"]
        window_start = max(0, tss - WINDOW_SIZE)
        window_end = tss + WINDOW_SIZE
        windows.append({
            "gene_id": row["gene_id"],
            "chromosome": row["chromosome"],
            "chromosome_id": row["chromosome_id"],
            "start": window_start,
            "end": window_end,
            "strand": row["strand"],
            "tss": tss
        })
    return pd.DataFrame(windows)

# Apply it to your extracted gene sequences
df_windows = compute_tss_window(df_genes)

# Preview the windowed TSS region data
print("\nTSS Window Regions:\n", df_windows.head())



TSS Window Regions:
                 gene_id chromosome chromosome_id   start     end strand  \
0  gene:Zm00001eb000010          1             1   24616   44616      +   
1  gene:Zm00001eb000020          1             1   36762   56762      -   
2  gene:Zm00001eb000030          1             1   96620  116620      -   
3  gene:Zm00001eb000040          1             1   98196  118196      -   
4  gene:Zm00001eb000050          1             1  104382  124382      -   

      tss  
0   34616  
1   46762  
2  106620  
3  108196  
4  114382  


In [3]:
from Bio import SeqIO
from Bio.Seq import Seq
import gzip
import pandas as pd

# Create a dictionary to store chromosome sequences
fasta_sequences = {}

# Load each fasta file and map its sequence by chromosome ID
for fasta_path in fasta_files:
    chrom_id = os.path.basename(fasta_path).split("chromosome.")[1].split(".")[0]
    if fasta_path.endswith(".gz"):
        with gzip.open(fasta_path, "rt") as handle:
            record = next(SeqIO.parse(handle, "fasta"))
    else:
        with open(fasta_path, "r") as handle:
            record = next(SeqIO.parse(handle, "fasta"))
    fasta_sequences[chrom_id] = record.seq

# Set window size (±10,000 bp = total 20,000)
WINDOW_SIZE = 10000
FIXED_LENGTH = 2 * WINDOW_SIZE

# Extract fixed-length TSS-centered sequences
def extract_window_sequence(row):
    chrom_id = str(row["chromosome_id"])
    chrom_seq = fasta_sequences.get(chrom_id, "")

    # Clip if chromosome not found
    if not chrom_seq:
        return "A" * FIXED_LENGTH

    start = row["start"]
    end = row["end"]

    # Extract the sequence window
    seq = chrom_seq[start:end]
    
    # Reverse complement if strand is negative
    if row["strand"] == "-":
        seq = seq.reverse_complement()
    
    # Pad or trim to exactly 20,000 bp
    if len(seq) < FIXED_LENGTH:
        seq += "A" * (FIXED_LENGTH - len(seq))  # pad with A
    elif len(seq) > FIXED_LENGTH:
        seq = seq[:FIXED_LENGTH]  # truncate

    return str(seq)

# Apply the function to the df_windows DataFrame
df_windows["sequence"] = df_windows.apply(extract_window_sequence, axis=1)

# Preview
print("\nFixed-length sequences extracted:\n", df_windows[["gene_id", "chromosome", "tss", "sequence"]].head())



Fixed-length sequences extracted:
                 gene_id chromosome     tss  \
0  gene:Zm00001eb000010          1   34616   
1  gene:Zm00001eb000020          1   46762   
2  gene:Zm00001eb000030          1  106620   
3  gene:Zm00001eb000040          1  108196   
4  gene:Zm00001eb000050          1  114382   

                                            sequence  
0  GTTTATTCGTTCGTTCGTCCGATCGTTCGATCGTTCATGGTTCGTT...  
1  CAATTATGTCATATGGGGTATAACTGTCTATTTACAAATGTAGATG...  
2  GGGTCGACGGGGCCGCGCGTGGCGCAGGCAGGATCGCCGGGGCCGC...  
3  GAGCGAGAGACGACGGGCTGCCTCTTGGCGGCCCAAGAGACGAGGT...  
4  TAGTGTTTGACACCCAAGGGGGGGGCTTGGTGCTGGCATGTTGTAG...  


In [4]:
df_windows.to_csv("tss_sequences.csv", index=False)

In [5]:
len(df_windows["sequence"].iloc[30])  # Check length of the first sequence


20000

In [6]:
import pandas as pd
import numpy as np

# Reuse your label encoder
def label_encode(sequence):
    mapping = {"A": 0, "C": 1, "G": 2, "T": 3}
    encoded = [mapping.get(base, 0) for base in sequence.upper()[:20000]]
    if len(encoded) < 20000:
        encoded += [0] * (20000 - len(encoded))  # pad if needed
    return encoded

# Load the DataFrame with sequences (already has 'sequence' column)
# Assuming you already have df_windows in memory
# If needed: df_windows = pd.read_csv("tss_sequences.csv")

# Prepare arrays
num_samples = len(df_windows)
X = np.zeros((num_samples, 20000), dtype=np.uint8)
gene_ids = []
chromosomes = []
tss_list = []

# Stream label encoding
for i, row in df_windows.iterrows():
    encoded = label_encode(row["sequence"])
    X[i] = encoded
    gene_ids.append(row["gene_id"])
    chromosomes.append(row["chromosome"])
    tss_list.append(row["tss"])

# Convert metadata to arrays
gene_ids = np.array(gene_ids)
chromosomes = np.array(chromosomes)
tss_list = np.array(tss_list)

# Save all in compressed format
np.savez_compressed("processed_data.npz", X=X, gene_id=gene_ids, chromosome=chromosomes, tss=tss_list)

print("✅ Memory-efficient data saved to 'processed_data.npz'")


✅ Memory-efficient data saved to 'processed_data.npz'


In [ ]:
df_windows1.head()

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Load from .npz file (created earlier)
data = np.load("processed_data.npz", allow_pickle=True)
X = data["X"]
gene_ids = data["gene_id"]
chromosomes = data["chromosome"]
tss_list = data["tss"]

# 2. Load df_genes with gene_id → start/end (from original CSV)
# Required for computing Y
df_meta = pd.read_csv("processed_data.csv", usecols=["gene_id", "start", "end"])
df_genes = df_meta.drop_duplicates(subset="gene_id")

# 3. Create gene_id → (start, end) mapping
gene_dict = {
    row["gene_id"]: (row["start"], row["end"])
    for _, row in df_genes.iterrows()
}

# 4. Generate Y
def generate_labels_for_window(window_start, gene_start, gene_end):
    labels = np.zeros(20000, dtype=int)
    gene_relative_start = max(0, gene_start - window_start)
    gene_relative_end = min(20000, gene_end - window_start)
    if gene_relative_start < gene_relative_end:
        labels[gene_relative_start:gene_relative_end] = 1
    return labels

Y = []
df_windows = pd.read_csv("processed_data.csv", usecols=["gene_id", "start"])
for i, row in df_windows.iterrows():
    gene_id = row["gene_id"]
    window_start = row["start"]
    if gene_id in gene_dict:
        gene_start, gene_end = gene_dict[gene_id]
        label = generate_labels_for_window(window_start, gene_start, gene_end)
    else:
        label = np.zeros(20000, dtype=int)
    Y.append(label)
Y = np.array(Y)

# 5. Split X and Y
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print("✅ Data loaded, labeled, and split.")


ValueError: Usecols do not match columns, columns expected but not found: ['start', 'end']

In [ ]:
from sklearn.model_selection import train_test_split

X = df_windows[[f"base_{i}" for i in range(20000)]].values.astype(np.int32)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
